In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten
#from utils import INPUT_SHAPE, batch_generator
import argparse
import os

from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 240, 640, 3
INPUT_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)



Using TensorFlow backend.


In [2]:
def batch_generator(data_dir, image_paths, steering_angles, batch_size, is_training):
    """
    Generate training image give image paths and associated steering angles
    """
    images = np.empty([batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS])
    steers = np.empty(batch_size)
    while True:
        i = 0
        for index in np.random.permutation(image_paths.shape[0]):
            center = image_paths[index]
            steering_angle = steering_angles[index]
            image = load_img(center)
            x = img_to_array(image)  
            x = x[240:480,:,:]
            image = array_to_img(x)
            # add the image and steering angle to the batch
            images[i] = image
            steers[i] = steering_angle
            i += 1
            if i == batch_size:
                break
        yield images, steers



In [3]:
data_df = pd.read_csv('/home/sumedh/Desktop/Hardware/DrivingLog.csv')

X = data_df['center'].values
y = data_df['steering'].values

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0)



In [4]:
#batch_generator('', X_train, y_train, 16, True)
X_train.shape

(8568,)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU, Input
from keras.layers import Conv2D, MaxPooling2D, Activation 
from keras.layers.convolutional import Convolution2D
from keras.models import Model
'''
model = Sequential()
model.add(Lambda(lambda x: x/127.5-1.0, input_shape=(240,640,3)))
model.add(Conv2D(16, 17, 17, activation='elu', subsample=(2, 2)))
model.add(Conv2D(36, 13, 13, activation='elu', subsample=(2, 2)))
model.add(Conv2D(48, 11, 11, activation='elu', subsample=(2, 2)))
model.add(Conv2D(64, 5, 5, activation='elu'))
model.add(Conv2D(64, 3, 3, activation='elu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100, activation='elu'))
model.add(Dense(50, activation='elu'))
model.add(Dense(10, activation='elu'))
model.add(Dense(1))
model.summary()

main_input = Input(shape=INPUT_SHAPE, dtype='float32', name='main_input')
lambda1 = Lambda(lambda x: x/127.5 - 1.,
            input_shape=INPUT_SHAPE,
            output_shape=INPUT_SHAPE)(main_input)
convShared1 = Convolution2D(16, 17, 17, subsample=(4, 4), border_mode="same")(lambda1)
eluShared1 = ELU()(convShared1)
convShared2 = Convolution2D(32, 11, 11, subsample=(2, 2), border_mode="same")(eluShared1)
eluShared2 = ELU()(convShared2)


SteeringConv = Convolution2D(64, 11, 11, subsample=(2, 2), border_mode="same")(eluShared2)
SteeringFlat = Flatten()(SteeringConv)
SteeringDrop1 = Dropout(.2)(SteeringFlat)
SteeringElu1 = ELU()(SteeringDrop1)
SteeringDense = Dense(512)(SteeringElu1)
SteeringDrop2 = Dropout(.5)(SteeringDense)
SteeringElu2 = ELU()(SteeringDrop2)
steering = Dense(1)(SteeringElu2)
model = Model(inputs=main_input, outputs= steering)
model.summary()
'''


model = Sequential()
model.add(Lambda(lambda x: x/127.5-1.0, input_shape=(240,640,3)))
model.add(Conv2D(32, kernel_size=(11, 11), strides=(1, 1),
                 activation='relu',
                 input_shape=INPUT_SHAPE))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
#model.add(Conv2D(32, (9, 9), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (13, 13), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (15, 15), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(1))
model.summary()


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 240, 640, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 230, 630, 32)      11648     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 115, 315, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 103, 303, 64)      346176    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 51, 151, 64)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 37, 137, 128)      1843328   
_________________________________________________________________
max_

In [ ]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.h5',
                             monitor='val_loss',
                             verbose=0,
                             save_best_only=True,
                             mode='auto')

model.compile(loss='mean_squared_error', optimizer='adam')

model.fit_generator(batch_generator('', X_train, y_train, 32, True),
                    250,
                    10,
                    max_q_size=1,
                    validation_data=batch_generator('', X_valid, y_valid, 16, False),
                    nb_val_samples=len(X_valid),
                    #callbacks=[checkpoint],
                    verbose=1)


Instructions for updating:
Use tf.cast instead.


/home/sumedh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  app.launch_new_instance()
/home/sumedh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., 250, 10, validation_data=<generator..., verbose=1, validation_steps=2856, max_queue_size=1)`
  app.launch_new_instance()


Epoch 1/10


In [ ]:
from keras.models import model_from_json

# serialize model to JSON
model_json = model.to_json()
with open("simple.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("simple.h5")
print("Saved model to disk")



In [ ]:
model.fit_generator(batch_generator('', X_train, y_train, 32, True),
                    250,
                    10,
                    max_q_size=1,
                    validation_data=batch_generator('', X_valid, y_valid, 16, False),
                    nb_val_samples=len(X_valid),
                    #callbacks=[checkpoint],
                    verbose=1)


In [ ]:
steers = []
a = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
counter = 0
for item in a:
    folderX = '/home/sumedh/Desktop/Hardware/Segmented/Run{}'.format(item)
    lenFolderX = len([f for f in os.listdir(folderX)])
    for i in range(lenFolderX):
        img = load_img('/home/sumedh/Desktop/Hardware/Segmented/Run{}/{}.jpg'.format(item, i+1))
        x = img_to_array(img)  
        x = x[240:480,:,:]
        image = array_to_img(x)
        image = np.asarray(image)       # from PIL image to numpy array

        image = np.array([image])       # the model expects 4D array
        #a = image[240:480,:,:]
        #print(image.shape)
        steering_angle = float(model.predict(image, batch_size=1))
        print(steering_angle)
        steers.append(steering_angle)

In [ ]:
import csv

actual = []
a = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
counter = 0
for item in a:
    folderX = '/home/sumedh/Desktop/Hardware/NewData/Run{}'.format(item)
    lenFolderX = len([f for f in os.listdir(folderX)])
    with open(folderX + "/" + 'Run{}.csv'.format(item),'rt')as f:
        data = csv.reader(f)
        for row in data:
            print(row[1])
            actual.append(float(row[1]))

In [ ]:
print(type(actual), type(steers))
print(type(actual[0]), type(steers[0]))


In [ ]:
from sklearn.metrics import r2_score
#actual = np.asarray(actual)       # from PIL image to numpy array
#steers = np.asarray(steers)       # from PIL image to numpy array
print(type(actual), type(steers))
print(r2_score(actual, steers))
